In [1]:
import re
from operator import sub

```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)
```

In [2]:
triplet = '{}=<\s*([-0-9]+),([-0-9]+),([-0-9]+)>'
re_str = (', ').join([
    triplet.format('p'),
    triplet.format('v'),
    triplet.format('a')
])
PARTICLE_RE = re.compile(re_str)

In [3]:
m = PARTICLE_RE.match('p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>')
assert m[7] == '-1'

In [4]:
tadd = lambda a, b: tuple(map(sum, zip(a, b)))
def tdist(a, b=(0, 0, 0)):
    return sum(map(abs, map(sub, a, b)))

In [5]:
assert tadd((1, 2, 3), (4, 5, 6)) == (5, 7, 9)
assert tdist((1, -2, 3)) == 6
assert tdist((1, -2, 3), (-1, 2, -3)) == 12

In [6]:
class Particle(object):
    def __init__(self, descriptor):
        self.parse_descriptor(descriptor)
        self.going_away = False
        
    def parse_descriptor(self, descriptor):
        m = PARTICLE_RE.match(descriptor)
        if not m:
            raise ValueError('Cannot parse descriptor: {}'.format(descriptor))
        self.position = (int(m[1]), int(m[2]), int(m[3]))
        self.velocity = (int(m[4]), int(m[5]), int(m[6]))
        self.acceleration = (int(m[7]), int(m[8]), int(m[9]))
    
    @property
    def acceleration_magnitude(self):
        return tdist(self.acceleration)

    @property
    def distance(self):
        return tdist(self.position)
           
    def move(self):
        prev_position = self.position
        self.velocity = tadd(self.velocity, self.acceleration)
        self.position = tadd(self.position, self.velocity)

        # Will we now be going further away from the origin
        # Are we accelerating?
        sign_diffs = [
            a
            for a, b in zip(self.acceleration, self.velocity)
            if abs(a + b) != abs(a) + abs(b)
        ]
        accelerating = False
        if not sign_diffs:
            accelerating = True

        # Are we further away from the origin than before?
        self.going_away = accelerating and (self.distance > tdist(prev_position))
        
    def __repr__(self):
        return 'p=<{}>, v=<{}>, a=<{}>'.format(
            str(self.position),
            str(self.velocity),
            str(self.acceleration)
        )        

In [7]:
with open('particles.txt') as fh:
    lines = fh.readlines()
lines = [l.strip() for l in lines]    

In [8]:
particles = [Particle(l) for l in lines]

In [9]:
particles[0]

p=<(1609, -863, -779)>, v=<(-15, 54, -69)>, a=<(-10, 0, 14)>

In [10]:
particles[0].acceleration_magnitude

24

In [11]:
accelerations = [Particle(l).acceleration_magnitude for l in lines]

In [12]:
accelerations.index(min(accelerations))

91

In [13]:
def play(particles):
    saved = 0
    survivors = particles
    
    while(survivors):
        # Collision detection
        seen = set()
        dups = set()
        num_survivors = len(survivors)
        for p in survivors:
            if p.position in seen:
                dups.add(p.position)
            else:
                seen.add(p.position)
        survivors = [s for s in survivors if s.position not in dups]
        
        if dups:
            print('COLLISION: removing:', num_survivors - len(survivors))

        # Can we save the particle with the highest acceleration?
        max_acc = max([s.acceleration_magnitude for s in survivors])
        max_dist = max([s.distance for s in survivors])        
        num_survivors = len(survivors)
        survivors = [
            s
            for s in survivors
            if s.acceleration_magnitude != max_acc or
            s.distance != max_dist or
            not s.going_away
        ]
        save_num = (num_survivors - len(survivors))
        if save_num:
            print('Saved: ', save_num)
        saved += save_num

        # Next position
        for s in survivors:
            s.move()
    
    return saved

In [14]:
particles = [Particle(l) for l in lines]
play(particles)

COLLISION: removing: 5
COLLISION: removing: 9
COLLISION: removing: 10
COLLISION: removing: 9
COLLISION: removing: 36
COLLISION: removing: 11
COLLISION: removing: 15
COLLISION: removing: 12
COLLISION: removing: 16
COLLISION: removing: 3
COLLISION: removing: 8
COLLISION: removing: 27
COLLISION: removing: 24
COLLISION: removing: 14
COLLISION: removing: 39
COLLISION: removing: 15
COLLISION: removing: 5
COLLISION: removing: 7
COLLISION: removing: 26
COLLISION: removing: 17
COLLISION: removing: 8
COLLISION: removing: 25
COLLISION: removing: 11
COLLISION: removing: 25
COLLISION: removing: 4
COLLISION: removing: 12
COLLISION: removing: 23
COLLISION: removing: 17
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved:  1
Saved: 

567